In [1]:
import pandas as pd
import numpy as np
import requests
import json

import matplotlib.pyplot as plt
from scipy.stats import linregress

from pathlib import Path
from census import Census
from api_keys import census_key

In [2]:
CA_cases_per_100_path = Path("Datasets/CA_adults_with_diabetes_per_100.csv")
CA_cases_per_100_data = pd.read_csv(CA_cases_per_100_path)

In [3]:
CA_cases_per_100_data.head()

,Geography,Year,Strata,Strata Name,Percent,Lower 95% CL,Upper 95% CL,Standard Error
0,California,2018,Total population,Total population,10.4,8.9,11.9,0.8
1,California,2018,Race-Ethnicity,White,8.4,6.9,9.9,0.8
2,California,2018,Race-Ethnicity,African-American,12.3,6.0,18.6,3.2
3,California,2018,Race-Ethnicity,Asian,8.5,3.2,13.9,2.7
4,California,2018,Race-Ethnicity,Hispanic,12.1,9.0,15.1,1.6


In [4]:
FRED_PPIC_Diabetes_path = Path("Datasets/FRED_PPIC_Diabetes.csv")
FRED_PPIC_Diabetes_data = pd.read_csv(FRED_PPIC_Diabetes_path)

In [5]:
FRED_PPIC_Diabetes_data.head()

,DATE,WPU06380103
0,2009-12-01,100.0
1,2010-01-01,99.8
2,2010-02-01,102.4
3,2010-03-01,103.0
4,2010-04-01,103.0


In [6]:
OWID_diabetes_vs_gdp_path = Path("Datasets/OWID_diabetes-prevalence-vs-gdp-per-capita.csv")
OWID_diabetes_vs_gdp_data = pd.read_csv(OWID_diabetes_vs_gdp_path)

In [7]:
OWID_diabetes_vs_gdp_data.head()

,Entity,Code,Year,Diabetes prevalence (% of population ages 20 to 79),"GDP per capita, PPP (constant 2017 international $)",Population (historical estimates),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,2011,7.6,1961.0963,29249156.0,NaN
2,Afghanistan,AFG,2021,10.9,1516.3057,40099460.0,NaN
3,Afghanistan,AFG,2002,NaN,1280.4631,21000258.0,NaN
4,Afghanistan,AFG,2003,NaN,1292.3335,22645136.0,NaN


In [8]:
weekly_deaths_countries_path = Path("Datasets/Weekly_Counts_of_Death_by_Diabetes.csv")
weekly_deaths_countries_data = pd.read_csv(weekly_deaths_countries_path)

In [9]:
weekly_deaths_countries_data.head()

,Jurisdiction,Week Ending Date,State Abbreviation,Year,Week,Cause Group,Number of Deaths,Cause Subgroup,Time Period,Suppress,Note,Average Number of Deaths in Time Period,Difference from 2015-2019 to 2020,Percent Difference from 2015-2019 to 2020,Type
0,Alabama,01/10/2015,AL,2015,1,Alzheimer disease and dementia,120,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)
1,Alabama,01/10/2015,AL,2015,1,Alzheimer disease and dementia,120,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Unweighted
2,Alabama,01/09/2016,AL,2016,1,Alzheimer disease and dementia,76,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)
3,Alabama,01/09/2016,AL,2016,1,Alzheimer disease and dementia,76,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Unweighted
4,Alabama,01/07/2017,AL,2017,1,Alzheimer disease and dementia,96,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)


In [10]:
population_variable = "B01003_001E"
income_variable = "B19013_001E"
employment_employed_variable = "B23025_004E"
poverty_variable = "B17001_002E"

In [11]:
url = f"https://api.census.gov/data/2019/acs/acs5?get=NAME,{population_variable},{income_variable},{employment_employed_variable},{poverty_variable}&for=state:*&key={census_key}"

census_data = requests.get(url).json()
print(json.dumps(census_data, indent=4, sort_keys=True))

[
    [
        "NAME",
        "B01003_001E",
        "B19013_001E",
        "B23025_004E",
        "B17001_002E",
        "state"
    ],
    [
        "Alabama",
        "4876250",
        "50536",
        "2097384",
        "795989",
        "01"
    ],
    [
        "Alaska",
        "737068",
        "77640",
        "347774",
        "76933",
        "02"
    ],
    [
        "Arizona",
        "7050299",
        "58945",
        "3130658",
        "1043764",
        "04"
    ],
    [
        "Arkansas",
        "2999370",
        "47597",
        "1303490",
        "496260",
        "05"
    ],
    [
        "California",
        "39283497",
        "75235",
        "18591241",
        "5149742",
        "06"
    ],
    [
        "Colorado",
        "5610349",
        "72331",
        "2904589",
        "565873",
        "08"
    ],
    [
        "Delaware",
        "957248",
        "68287",
        "455620",
        "109400",
        "10"
    ],
    [
        "District of Colu

## Prep Census and Merge with Diabetes Death Info

In [12]:
weekly_deaths_countries_data.head()

,Jurisdiction,Week Ending Date,State Abbreviation,Year,Week,Cause Group,Number of Deaths,Cause Subgroup,Time Period,Suppress,Note,Average Number of Deaths in Time Period,Difference from 2015-2019 to 2020,Percent Difference from 2015-2019 to 2020,Type
0,Alabama,01/10/2015,AL,2015,1,Alzheimer disease and dementia,120,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)
1,Alabama,01/10/2015,AL,2015,1,Alzheimer disease and dementia,120,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Unweighted
2,Alabama,01/09/2016,AL,2016,1,Alzheimer disease and dementia,76,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)
3,Alabama,01/09/2016,AL,2016,1,Alzheimer disease and dementia,76,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Unweighted
4,Alabama,01/07/2017,AL,2017,1,Alzheimer disease and dementia,96,Alzheimer disease and dementia,2015-2019,NaN,NaN,103,NaN,NaN,Predicted (weighted)


In [13]:
#extract data where "Cause Subgroup" is "Diabetes" and "Type" is "Unweighted"
#if we don't specify "Type" there will be duplicates
weekly_diabetes_deaths = weekly_deaths_countries_data.loc[(weekly_deaths_countries_data["Cause Subgroup"] == "Diabetes") & (weekly_deaths_countries_data["Type"] == "Unweighted")]

#extract only the relevant columns
weekly_diabetes_deaths = weekly_diabetes_deaths[["Jurisdiction", "Week Ending Date", "Number of Deaths", "Cause Subgroup"]]

weekly_diabetes_deaths.head()

,Jurisdiction,Week Ending Date,Number of Deaths,Cause Subgroup
285699,Alabama,01/10/2015,36,Diabetes
285701,Alabama,01/09/2016,32,Diabetes
285703,Alabama,01/07/2017,25,Diabetes
285705,Alabama,01/06/2018,34,Diabetes
285707,Alabama,01/05/2019,22,Diabetes


In [14]:
#convert "Week Ending Date" into datetime
#make new columns of month and year
weekly_diabetes_deaths["Month"] = pd.to_datetime(weekly_diabetes_deaths["Week Ending Date"]).dt.month
weekly_diabetes_deaths["Year"] = pd.to_datetime(weekly_diabetes_deaths["Week Ending Date"]).dt.year

#remove data from after 2020 (the census data ends in 2020)
weekly_diabetes_deaths = weekly_diabetes_deaths.loc[weekly_diabetes_deaths["Year"] <= 2020]

In [15]:
#sum deaths by state and year
weekly_diabetes_deaths = pd.DataFrame({
    "Number of Diabetes Deaths":weekly_diabetes_deaths.groupby(["Jurisdiction", "Year"])["Number of Deaths"].sum()
})

weekly_diabetes_deaths = weekly_diabetes_deaths.reset_index()

weekly_diabetes_deaths

,Jurisdiction,Year,Number of Diabetes Deaths
0,Alabama,2015,1139
1,Alabama,2016,1161
2,Alabama,2017,1133
3,Alabama,2018,1133
4,Alabama,2019,1196
...,...,...,...
280,Wisconsin,2016,1446
281,Wisconsin,2017,1420
282,Wisconsin,2018,1518
283,Wisconsin,2019,1537


In [16]:
census_column_names = ["State", "Year", "Population", "Median Household Income", "Number of People Employed", "Number of People in Poverty"]
census_df = pd.DataFrame(columns = census_column_names)

census_df

,State,Year,Population,Median Household Income,Number of People Employed,Number of People in Poverty


In [17]:
#extract years in the weekly deaths data
#will be used to extract census data by year
years_list = weekly_diabetes_deaths["Year"].unique()
years_list = list(years_list)

years_list

[2015, 2016, 2017, 2018, 2019, 2020]

In [18]:
#list will become column in census_df
name_values = []
population_values = []
income_values = []
employment_values = []
poverty_values = []
year_values = []

#loop through the years we have of weekly deaths data
#for each year fetch the census data
for year in years_list:
    
    url = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,{population_variable},{income_variable},{employment_employed_variable},{poverty_variable}&for=state:*&key={census_key}"
    census_data_by_year = requests.get(url).json()
    
    #loop through each state
    #add state data to lists
    for i in range(0, 51):
        state_name = census_data_by_year[i][0]
        name_values.append(state_name)
    
        state_population = census_data_by_year[i][1]
        population_values.append(state_population)
    
        state_income = census_data_by_year[i][2]
        income_values.append(state_income)
    
        state_employment = census_data_by_year[i][3]
        employment_values.append(state_employment)
    
        state_poverty = census_data_by_year[i][4]
        poverty_values.append(state_poverty)
        
        year_values.append(year)
    
    print(f"{year} data complete")

#put the list into relevant column
census_df["State"] = name_values
census_df["Year"] = year_values
census_df["Population"] = population_values
census_df["Median Household Income"] = income_values
census_df["Number of People Employed"] = employment_values
census_df["Number of People in Poverty"] = poverty_values

2015 data complete
2016 data complete
2017 data complete
2018 data complete
2019 data complete
2020 data complete


In [19]:
census_df.head()

,State,Year,Population,Median Household Income,Number of People Employed,Number of People in Poverty
0,NAME,2015,B01003_001E,B19013_001E,B23025_004E,B17001_002E
1,Mississippi,2015,2988081,39665,1203914,651545
2,Missouri,2015,6045448,48173,2807996,915972
3,Montana,2015,1014699,47169,485446,150586
4,Nebraska,2015,1869365,52997,968134,231115


In [20]:
#remove Wyoming data since it's not in the weekly deaths data

census_df = census_df.loc[(census_df["State"] != "NAME") & (census_df["State"] != "Wyoming")]
census_df.head()

,State,Year,Population,Median Household Income,Number of People Employed,Number of People in Poverty
1,Mississippi,2015,2988081,39665,1203914,651545
2,Missouri,2015,6045448,48173,2807996,915972
3,Montana,2015,1014699,47169,485446,150586
4,Nebraska,2015,1869365,52997,968134,231115
5,Nevada,2015,2798636,51847,1267312,426337


In [21]:
#change "Jurisdiction" column to be named "State"
weekly_diabetes_deaths = weekly_diabetes_deaths.rename(columns = {"Jurisdiction":"State"})
weekly_diabetes_deaths.head()

,State,Year,Number of Diabetes Deaths
0,Alabama,2015,1139
1,Alabama,2016,1161
2,Alabama,2017,1133
3,Alabama,2018,1133
4,Alabama,2019,1196


In [22]:
#merge weekly deaths data and census data by "State" and "Year"
weekly_deaths_and_census = pd.merge(weekly_diabetes_deaths, census_df, how = 'inner', left_on = ['State', 'Year'], right_on = ['State', 'Year'])
weekly_deaths_and_census

,State,Year,Number of Diabetes Deaths,Population,Median Household Income,Number of People Employed,Number of People in Poverty
0,Alabama,2015,1139,4830620,43623,2022325,887260
1,Alabama,2016,1161,4841164,44758,2042025,868666
2,Alabama,2017,1133,4850771,46472,2055509,849699
3,Alabama,2018,1133,4864680,48486,2076708,829400
4,Alabama,2019,1196,4876250,50536,2097384,795989
...,...,...,...,...,...,...,...
257,Wisconsin,2016,1446,5754798,54610,2910339,713472
258,Wisconsin,2017,1420,5763217,56759,2939880,692719
259,Wisconsin,2018,1518,5778394,59209,2964540,668220
260,Wisconsin,2019,1537,5790716,61747,2982359,639160


In [23]:
#change to numeric instead of string so we can perform operations
weekly_deaths_and_census[["Year", "Number of Diabetes Deaths", "Population", "Median Household Income", "Number of People Employed", "Number of People in Poverty"]] = weekly_deaths_and_census[["Year", "Number of Diabetes Deaths", "Population", "Median Household Income", "Number of People Employed", "Number of People in Poverty"]].astype(float)

In [24]:
weekly_deaths_and_census["Number of People Employed"] = (weekly_deaths_and_census["Number of People Employed"] / weekly_deaths_and_census["Population"]) * 100
weekly_deaths_and_census["Number of People in Poverty"] = (weekly_deaths_and_census["Number of People in Poverty"] / weekly_deaths_and_census["Population"]) * 100
weekly_deaths_and_census["Number of Diabetes Deaths"] = (weekly_deaths_and_census["Number of Diabetes Deaths"] / weekly_deaths_and_census["Population"]) * 100

In [25]:
weekly_deaths_and_census = weekly_deaths_and_census.rename(columns = {"Number of Diabetes Deaths":"Percentage of Population Dead by Diabetes", "Number of People Employed": "Percentage of Population Employed", "Number of People in Poverty":"Percentage of Population in Poverty"})
weekly_deaths_and_census

,State,Year,Percentage of Population Dead by Diabetes,Population,Median Household Income,Percentage of Population Employed,Percentage of Population in Poverty
0,Alabama,2015.0,0.023579,4830620.0,43623.0,41.864709,18.367415
1,Alabama,2016.0,0.023982,4841164.0,44758.0,42.180455,17.943329
2,Alabama,2017.0,0.023357,4850771.0,46472.0,42.374893,17.516782
3,Alabama,2018.0,0.023290,4864680.0,48486.0,42.689509,17.049426
4,Alabama,2019.0,0.024527,4876250.0,50536.0,43.012233,16.323794
...,...,...,...,...,...,...,...
257,Wisconsin,2016.0,0.025127,5754798.0,54610.0,50.572392,12.397863
258,Wisconsin,2017.0,0.024639,5763217.0,56759.0,51.011093,12.019658
259,Wisconsin,2018.0,0.026270,5778394.0,59209.0,51.303874,11.564113
260,Wisconsin,2019.0,0.026542,5790716.0,61747.0,51.502422,11.037668


## Does diabetes death correlate with poverty/GDP/population, employment?

NOTE: some states are missing (ex: Alaska) bc they don't have weekly diabetes deaths data

## Is California an outlier?

In [26]:
CA_census_and_deaths_data = weekly_deaths_and_census.loc[weekly_deaths_and_census["State"] == "California"]

CA_census_and_deaths_data

,State,Year,Percentage of Population Dead by Diabetes,Population,Median Household Income,Percentage of Population Employed,Percentage of Population in Poverty
18,California,2015.0,0.022472,38421464.0,61818.0,44.887306,15.968007
19,California,2016.0,0.023915,38654206.0,63783.0,45.472780,15.533257
20,California,2017.0,0.024560,38982847.0,67169.0,46.158545,14.810124
21,California,2018.0,0.024325,39148760.0,71228.0,46.767795,14.016130
22,California,2019.0,0.025016,39283497.0,75235.0,47.325830,13.109174
23,California,2020.0,0.029213,39346023.0,78672.0,47.392068,12.335259


In [27]:
average_deaths = list(weekly_deaths_and_census.groupby("Year")["Percentage of Population Dead by Diabetes"].mean())
average_population = list(weekly_deaths_and_census.groupby("Year")["Population"].mean())
average_income = list(weekly_deaths_and_census.groupby("Year")["Median Household Income"].mean())
average_employed = list(weekly_deaths_and_census.groupby("Year")["Percentage of Population Employed"].mean())
average_poverty = list(weekly_deaths_and_census.groupby("Year")["Percentage of Population in Poverty"].mean())

In [30]:
averages_data = pd.DataFrame({
    "Year": [2015, 2016, 2017, 2018, 2019, 2020],
    "Average Percentage of Diabetes Deaths":average_deaths,
    "Average Population":average_population, 
    "Average Median Household Income":average_income, 
    "Average Percentage of Population Employed":average_employed, 
    "Average Percentage of Population in Poverty":average_poverty
})

averages_data

,Year,Average Percentage of Diabetes Deaths,Average Population,Average Median Household Income,Average Percentage of Population Employed,Average Percentage of Population in Poverty
0,2015,0.023243,7.097218e+06,52221.119048,45.889859,15.654588
1,2016,0.024146,7.013954e+06,54147.325581,46.214550,15.209208
2,2017,0.025103,7.158523e+06,55509.883721,46.575400,14.906538
3,2018,0.024875,6.941314e+06,59079.545455,47.080720,14.349018
4,2019,0.024617,7.251913e+06,62443.113636,47.808913,12.883947
5,2020,0.028841,6.946276e+06,63613.478261,47.593793,12.952921
